In [6]:
from openai import OpenAI
import os

In [7]:
openai_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=openai_key)

In [8]:
def create_test_prompt(topic,num_questions,num_possible_answer):
    
    prompt = f"""Create a multiple-choice quiz on the topic of {topic}. The quizz should contain {num_questions} questions. 
    Each question should have {num_possible_answer} as options. Include the Correct answer after every question.
    Only Include question which are based on Actual Facts. Questions should be PHD Level.
    Explore many different subtopics from the {topic}.
    
    
                    
    Example: 1

    1. Which of the following statements about quantum entanglement is true?
    a) Entanglement can be explained solely through classical mechanics.
    b) Entanglement is a phenomenon only observed at the macroscopic level.
    c) Entanglement violates the principle of locality.
    d) Entanglement occurs only between particles with opposite spin.
    
    
    Answer: c) Entanglement violates the principle of locality.
    
    
    Example: 2
    
    2. What is the primary function of microRNAs (miRNAs) in gene expression regulation?
    a) Promote translation of mRNA into protein.
    b) Inhibit transcription of DNA into mRNA.
    c) Degrade mRNA molecules.
    d) Inhibit translation of mRNA into protein.
    
    
    Answer: d) Inhibit translation of mRNA into protein.
    
    
    Example: 3
    
    3. Computer Science:In the context of machine learning, what does the term "overfitting" refer to?
    a) The model's inability to capture the underlying trend in the data.
    b) The model performing well on the training data but poorly on unseen data.
    c) The model's underestimation of the complexity of the data.
    d) The model's tendency to generalize well to unseen data.


    Answer: b) The model performing well on the training data but poorly on unseen data.
    
    
    """
    return prompt

In [9]:
prompt = create_test_prompt("chemistry",4,1)

In [10]:

response = client.chat.completions.create(
    model = 'gpt-3.5-turbo',
    messages=[
        {
            "role":"system","content":"""You're an experienced quizmaster tasked with crafting questions and answers 
            tailored to the user's specifications. The questions should be at the level of Phd.
            """,
            "role":"user","content":f"{prompt}"
        }
    ],
    temperature = 0.9
    
)

In [11]:
output = response.choices[0].message.content

In [12]:
# print(output)

In [13]:
questions = {1:''}
question_number = 1
question_block = output.split("\n")
question_block = [block.strip() for block in question_block]
for block in question_block:
    if not block.startswith("Answer: "):
        questions[question_number] += block+'\n'
    else:
        if question_number < 4:
            question_number += 1
            questions[question_number] = ''
print(questions)
        

{1: '1. Which of the following elements is classified as a noble gas in the periodic table?\na) Neon\nb) Sodium\nc) Phosphorus\nd) Calcium\n\n', 2: '\n\n2. What is the empirical formula of a compound containing 40.0% carbon, 6.7% hydrogen, and 53.3% oxygen by mass?\na) CH\nb) CHO2\nc) C5H4O3\nd) C4H6O2\n\n', 3: '\n\n3. Which of the following statements about the Arrhenius equation is true?\na) It describes the relationship between reactants and products in a chemical reaction.\nb) It quantifies the rate of a chemical reaction based on reaction temperature.\nc) It predicts the concentration of reactants at equilibrium.\nd) It was proposed to explain the behavior of ideal gases.\n\n', 4: '\n\n4. Which of the following elements has the highest electronegativity value?\na) Hydrogen\nb) Fluorine\nc) Sodium\nd) Nitrogen\n\n'}


In [14]:
def create_student_view(test,num_question):
    questions = {1:''}
    question_number = 1
    question_block = test.split("\n\n")
    question_block = [block.strip() for block in question_block]
    for block in question_block:
        if not block.startswith("Answer: "):
            questions[question_number] += block+'\n'
        else:
            if question_number < num_question:
                question_number += 1
                questions[question_number] = ''
    return questions
        

In [15]:
answer = {1:''}
question_number = 1
question_block = output.split("\n")
question_block = [block.strip() for block in question_block]
for block in question_block:
    if  block.startswith("Answer: "):
        answer[question_number] +=  block+'\n'
        if question_number < 4:
            question_number += 1
            answer[question_number] = ''
print(answer)

{1: 'Answer: a) Neon\n', 2: 'Answer: d) C4H6O2\n', 3: 'Answer: b) It quantifies the rate of a chemical reaction based on reaction temperature.\n', 4: 'Answer: b) Fluorine\n'}


In [16]:
def extract_answer(test,num_question):
    answer = {1:''}
    question_number = 1
    question_block = test.split("\n\n")
    for block in question_block:
        if  block.startswith("Answer: "):
            answer[question_number] +=  block+'\n'
            if question_number < num_question:
                question_number += 1
                answer[question_number] = ''
    return answer
        

In [17]:
student_view = create_student_view(output,4)
answer_key = extract_answer(output,4)

In [18]:
answer_key

{1: ''}

In [19]:
# Take the exam

def take(student_view):
    
    student_answer = {}
    for question, question_view in student_view.items():
        print(question_view)
        answer = input("Enter Your Answer: ")
        student_answer[question] = answer
    return student_answer

In [20]:
std_answers = take(student_view)

1. Which of the following elements is classified as a noble gas in the periodic table?
    a) Neon
    b) Sodium
    c) Phosphorus
    d) Calcium
    
    Answer: a) Neon
2. What is the empirical formula of a compound containing 40.0% carbon, 6.7% hydrogen, and 53.3% oxygen by mass?
    a) CH
    b) CHO2
    c) C5H4O3
    d) C4H6O2
    
    Answer: d) C4H6O2
3. Which of the following statements about the Arrhenius equation is true?
    a) It describes the relationship between reactants and products in a chemical reaction.
    b) It quantifies the rate of a chemical reaction based on reaction temperature.
    c) It predicts the concentration of reactants at equilibrium.
    d) It was proposed to explain the behavior of ideal gases.
    
    Answer: b) It quantifies the rate of a chemical reaction based on reaction temperature.
4. Which of the following elements has the highest electronegativity value?
    a) Hydrogen
    b) Fluorine
    c) Sodium
    d) Nitrogen
    
    Answer: b) Fluo

In [1]:
score = 0
l = len(answer_key)
for question,answer in answer_key.items():
    ans  = answer.split(":")[1].strip()[0]
    if std_answers[question].lower() == ans.lower():
        score += 1
print(f"You Sored {score} out of {l} !!!")

NameError: name 'answer_key' is not defined

In [340]:
s = "Sociology: Which sociological perspective emphasizes the importance of social structures and institutions in shaping individuals' behaviors and choices? a) Symbolic interactionism b) Feminist theory"

In [343]:
s.split("? ")[1]

'a) Symbolic interactionism b) Feminist theory'

In [23]:
student_answers = {
"1":"b) Proton",
"2":"c) The universe will reach a state of maximum entropy.",
"3":"a) The speed of light in a vacuum",
"4":"b) Solar wind"
}

answer_key = {
"1":"Answer: a) Electron",
"2":"Answer: c) The universe will reach a state of maximum entropy.",
"3":"Answer: a) The speed of light in a vacuum",
"4":"Answer: c) Earth's rotation"
}

In [42]:
for q,a in student_answers.items():
    student_answers[q] = a.split(')')[0]
print(student_answers)

for q,a in answer_key.items():
    answer_key[q] = a.split('Answer:')[1].strip()[0]
print(answer_key)

{'1': 'b', '2': 'c', '3': 'a', '4': 'b'}
{'1': 'a', '2': 'c', '3': 'a', '4': 'c'}


In [24]:
for question,answer in answer_key.items():
    ans  = answer.split('Answer:')[1].strip()[0]
    if student_answers[question].lower() == ans.lower():
        score += 1
print(score)

In [26]:
calculate_score(student_answers,answer_key)

0